# FARS Multi-Sensory Database - Exploratory Analysis

This notebook demonstrates basic analysis of the pedestrian crash database.

**Topics covered:**
- Database connection
- Data quality assessment
- Geographic visualization
- Environmental factor analysis
- Multi-sensory crash profiling

In [ ]:
# Setup
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from datetime import datetime, timedelta

# Project imports
from src.database import Database
from src.utils.geo_utils import calculate_distance
from src.utils.validation import DataQualityChecker

# Plotting setup
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Setup complete")

## 1. Database Connection

In [ ]:
# Connect to database
db = Database()

if db.test_connection():
    print("✓ Database connection successful")
else:
    print("✗ Database connection failed")
    raise Exception("Cannot connect to database")

## 2. Data Overview

In [ ]:
# Get summary statistics from database
summary = db.execute("SELECT * FROM vw_database_summary;")

if summary:
    summary_df = pd.DataFrame(
        [summary[0]],
        columns=[
            'total_crashes', 'states_covered', 'counties_covered',
            'crashes_with_streetview', 'crashes_with_sound',
            'crashes_with_air_quality', 'crashes_with_weather',
            'streetview_coverage_pct', 'sound_coverage_pct',
            'aq_coverage_pct', 'weather_coverage_pct'
        ]
    ).T
    summary_df.columns = ['Value']
    print("Database Summary:")
    print(summary_df)
else:
    print("No data in database yet")

In [ ]:
# Load crash data
query = """
SELECT 
    crash_id,
    state,
    city,
    county,
    latitude,
    longitude,
    crash_date,
    crash_time,
    time_of_day,
    severity,
    vehicle_speed
FROM crashes
LIMIT 1000;
"""

crashes = pd.read_sql(query, db._engine)
print(f"Loaded {len(crashes):,} crashes")
crashes.head()

## 3. Data Quality Assessment

In [ ]:
# Check data completeness
completeness_query = """
SELECT 
    has_streetview,
    has_sound,
    has_air_quality,
    has_weather,
    has_lighting,
    has_analysis,
    completeness_percentage
FROM vw_data_completeness
ORDER BY completeness_percentage DESC
LIMIT 100;
"""

completeness = pd.read_sql(completeness_query, db._engine)

# Plot completeness distribution
plt.figure(figsize=(10, 6))
plt.hist(completeness['completeness_percentage'], bins=20, edgecolor='black')
plt.xlabel('Completeness Percentage')
plt.ylabel('Number of Crashes')
plt.title('Data Completeness Distribution')
plt.axvline(completeness['completeness_percentage'].mean(), 
            color='red', linestyle='--', label=f'Mean: {completeness["completeness_percentage"].mean():.1f}%')
plt.legend()
plt.show()

print(f"Average completeness: {completeness['completeness_percentage'].mean():.1f}%")

## 4. Geographic Analysis

In [ ]:
# Crashes by state
crashes_by_state = crashes['state'].value_counts()

plt.figure(figsize=(12, 6))
crashes_by_state.head(10).plot(kind='bar')
plt.xlabel('State')
plt.ylabel('Number of Crashes')
plt.title('Top 10 States by Pedestrian Crashes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Interactive map of crashes (sample)
sample_crashes = crashes.dropna(subset=['latitude', 'longitude']).sample(min(100, len(crashes)))

# Center map on mean coordinates
center_lat = sample_crashes['latitude'].mean()
center_lon = sample_crashes['longitude'].mean()

# Create map
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add crash markers
for idx, crash in sample_crashes.iterrows():
    folium.CircleMarker(
        location=[crash['latitude'], crash['longitude']],
        radius=5,
        popup=f"{crash['crash_id']}<br>{crash['city']}<br>{crash['crash_date']}",
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.6
    ).add_to(m)

# Display map
m

## 5. Temporal Analysis

In [ ]:
# Crashes by time of day
time_of_day_counts = crashes['time_of_day'].value_counts()

plt.figure(figsize=(8, 6))
time_of_day_counts.plot(kind='bar', color=['#FFD700', '#FF6347', '#4169E1', '#2F4F4F'])
plt.xlabel('Time of Day')
plt.ylabel('Number of Crashes')
plt.title('Crashes by Time of Day')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Crashes by hour (if crash_time available)
crashes_with_time = crashes.dropna(subset=['crash_time'])

if len(crashes_with_time) > 0:
    # Extract hour from crash_time
    crashes_with_time['hour'] = pd.to_datetime(crashes_with_time['crash_time'], format='%H:%M:%S').dt.hour
    
    hour_counts = crashes_with_time['hour'].value_counts().sort_index()
    
    plt.figure(figsize=(12, 6))
    plt.plot(hour_counts.index, hour_counts.values, marker='o', linewidth=2, markersize=8)
    plt.xlabel('Hour of Day')
    plt.ylabel('Number of Crashes')
    plt.title('Crashes by Hour of Day')
    plt.xticks(range(0, 24))
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No crash time data available")

## 6. Multi-Sensory Analysis (If Data Available)

In [ ]:
# Load multi-sensory data for crashes
multisensory_query = """
SELECT 
    crash_id,
    city,
    state,
    mean_loudness_db,
    pm2_5,
    aqi_category,
    temperature_f,
    lighting_condition,
    cv_has_crosswalk,
    environmental_stress_score,
    pedestrian_safety_score
FROM vw_crash_complete
WHERE data_complete = TRUE
LIMIT 100;
"""

try:
    multisensory = pd.read_sql(multisensory_query, db._engine)
    
    if len(multisensory) > 0:
        print(f"Loaded {len(multisensory):,} crashes with complete multi-sensory data")
        
        # Display first few rows
        display(multisensory.head())
        
        # Correlation matrix
        numeric_cols = ['mean_loudness_db', 'pm2_5', 'temperature_f', 
                       'environmental_stress_score', 'pedestrian_safety_score']
        
        available_cols = [col for col in numeric_cols if col in multisensory.columns]
        
        if len(available_cols) > 1:
            plt.figure(figsize=(10, 8))
            sns.heatmap(
                multisensory[available_cols].corr(),
                annot=True,
                cmap='coolwarm',
                center=0,
                square=True
            )
            plt.title('Multi-Sensory Factor Correlation Matrix')
            plt.tight_layout()
            plt.show()
    else:
        print("No complete multi-sensory data available yet")
        print("Run data collectors to populate environmental data")
        
except Exception as e:
    print(f"Multi-sensory data not available: {e}")
    print("This is normal if you haven't run the data collectors yet")

## 7. High-Risk Location Analysis

In [ ]:
# Find high-risk crashes (if analysis data available)
high_risk_query = """
SELECT 
    crash_id,
    intersection,
    city,
    state,
    risk_factor_count,
    environmental_stress_score,
    pedestrian_safety_score
FROM vw_high_risk_crashes
ORDER BY risk_factor_count DESC, environmental_stress_score DESC
LIMIT 20;
"""

try:
    high_risk = pd.read_sql(high_risk_query, db._engine)
    
    if len(high_risk) > 0:
        print(f"Found {len(high_risk)} high-risk crash locations")
        display(high_risk)
        
        # Visualize risk factors
        plt.figure(figsize=(10, 6))
        plt.scatter(
            high_risk['environmental_stress_score'],
            high_risk['pedestrian_safety_score'],
            s=high_risk['risk_factor_count'] * 50,
            alpha=0.6,
            c=high_risk['risk_factor_count'],
            cmap='YlOrRd'
        )
        plt.xlabel('Environmental Stress Score')
        plt.ylabel('Pedestrian Safety Score')
        plt.title('High-Risk Crashes: Stress vs Safety\n(Size = Number of Risk Factors)')
        plt.colorbar(label='Risk Factor Count')
        plt.tight_layout()
        plt.show()
    else:
        print("No high-risk crashes identified yet")
        
except Exception as e:
    print(f"High-risk analysis not available: {e}")

## 8. Export Results

In [ ]:
# Export crash data to CSV
output_path = project_root / 'outputs' / 'exports' / 'crash_sample.csv'
output_path.parent.mkdir(parents=True, exist_ok=True)

crashes.to_csv(output_path, index=False)
print(f"✓ Exported {len(crashes):,} crashes to {output_path}")

## 9. Next Steps

To get more insights:

1. **Collect environmental data:**
   ```bash
   python scripts/03_download_streetview.py --limit 10
   python scripts/05_collect_air_quality.py --limit 10
   python scripts/06_collect_weather.py --limit 10
   ```

2. **Run computer vision analysis:**
   ```bash
   python scripts/07_analyze_images.py
   ```

3. **Generate comprehensive reports:**
   ```bash
   python scripts/08_generate_reports.py
   ```

4. **Explore specific analyses:**
   - Environmental justice by county
   - Infrastructure deficiency patterns
   - Crash clustering and hotspots
   - Weather/lighting correlations

In [ ]:
# Clean up
db.close()
print("✓ Analysis complete")